### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu", "La", "Li", "Tm", "Ti", "Te", "Tb", "Tc", "Yb", "Dy",\
#"I", "Y", "Ag", "Al", "As", "In"]

headers = ["H","Li","Be","B","C","N","O","F","Na","Mg","Al","Si","P","S","Cl",\
"K","Ca","Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga",\
"Ge","As","Se","Br","Rb","Sr","Y","Zr","Nb","Mo",\
"Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",\
"Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr",\
"Nd","Sm","Eu","Gd","Tb","Dy","Ho","Er",\
"Tm","Yb","Lu","Hf","Ta","W","Re","Os","Ir",\
"Pt","Au","Hg","Tl","Pb","Bi","Ac","Th","Pa",\
"U","Np","Am"] #hofmann

#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu"]


unitformula = pd.read_csv("scaledformulae.csv", usecols=headers)
#unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [ ]:
from sklearn import linear_model, svm, kernel_ridge, ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(unitformula, vols, random_state = 24)

### Main algorithm

In [ ]:
#reg = ensemble.GradientBoostingRegressor()
#reg.fit(X_train, y_train)

In [ ]:
estimator = [(linear_model.RANSACRegressor(linear_model.LinearRegression())),\
             (linear_model.RANSACRegressor(linear_model.Ridge(alpha=100))), (RandomForestRegressor()),\
             (ensemble.GradientBoostingRegressor()), (ensemble.ExtraTreesRegressor())]
for reg in estimator:
    #X_train, X_test, y_train, y_test = train_test_split(unitformula, vols, random_state = 24)
    reg.fit(X_train, y_train)
    pdv = reg.predict(X_test)
    print "Using: ", reg
    print reg.score(X_test,y_test)
    pdvd = pd.DataFrame(pdv, columns = ["Pred Vol"])
    ytest = pd.DataFrame(y_test)
    ytest = ytest.reset_index(drop=True)
    pdvd["Pred Vol"].div(ytest["Volume"]).hist(bins=50)
    print pdvd["Pred Vol"].div(ytest["Volume"]).describe()
    plt.show()

In [ ]:
hofmw = np.array([37.3, 42, 31.2, 36, 13.24, 66, 32.7, 5.08, 29.5, 41.6, 56, 37.8, 57,\
13.87, 35, 51, 42, 36, 36, 38, 31.9, 11.39, 25.2, 39, 53, 27, 54, 26,\
26, 37, 50, 30.4, 13.24, 11.17, 47, 11.8, 37.3, 52.8, 50, 24, 42, 48, 30.3,\
29.4, 25.8, 45, 54, 45, 46, 28.1, 26.9, 58, 22.6, 49,\
27.3, 46.7, 45, 38, 59, 50, 46.2, 44, 35, 39.6, 36.4, 55])
elements = unitformula.columns.values
X3 = pd.DataFrame()
for line in zip(elements, hofmw):
        X3[line[0]] = unitformula[line[0]].apply(lambda x: x*line[1])
#X2["sumhoff"] = X2.sum(axis=1)
hoffpred = X3.sum(axis=1)

In [ ]:
pdvd.to_csv("ourvols.csv")
hoffpred.to_csv("hoffvols.csv")

This page is the same as the loop attempts before, upon deeper inspection is was noticed that 
Something seems off about volumes.csv, multiple values repeat but when cross-checked with the database these values aren't the same for many enteries.
New sets of csv files should be produced.